## Advanced Dataframes (review and practice)

In [67]:
import pandas as pd
import numpy as np
from pydataset import data
import matplotlib.pyplot as plt
import seaborn as sns

## Exercise I

1. Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [2]:
def get_db_url(host, user, password, database):
    url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
    return url

In [3]:
from env import host, user, password
url = get_db_url(host, user, password, 'employees')

In [4]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


4. Read the employees and titles tables into two separate DataFrames.

In [5]:
titles_df = pd.read_sql('SELECT * FROM titles', url)
titles_df.sample(5)

,emp_no,title,from_date,to_date
159747,207844,Staff,1996-04-04,9999-01-01
170758,215286,Senior Staff,1987-03-31,1995-02-14
86470,68403,Engineer,1986-05-17,1994-05-17
203730,237663,Assistant Engineer,1997-12-24,9999-01-01
40176,37184,Assistant Engineer,1987-07-26,1992-07-25


In [6]:
employees_df = pd.read_sql('SELECT * FROM employees', url)
employees_df.sample(5)

,emp_no,birth_date,first_name,last_name,gender,hire_date
109164,209140,1954-08-30,Mabhin,Maksimenko,F,1991-05-31
229806,429782,1955-02-25,Jaewon,Sundgren,F,1991-08-22
255523,455499,1958-06-05,Moni,McDermid,M,1993-04-03
132704,232680,1955-12-05,Gennadi,Baba,F,1992-10-27
73487,83488,1953-03-30,Uzi,Gente,F,1989-10-28


5. How many rows and columns do you have in each DataFrame? 

In [7]:
titles_df.shape

(443308, 4)

In [8]:
employees_df.shape

(300024, 6)

6. Display the summary statistics for each DataFrame.

In [9]:
titles_df.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


In [10]:
employees_df.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


7. How many unique titles are in the titles DataFrame?

In [11]:
titles_df.title.value_counts() # or
titles_df['title'].value_counts()

Engineer              115003
Staff                 107391
Senior Engineer        97750
Senior Staff           92853
Technique Leader       15159
Assistant Engineer     15128
Manager                   24
Name: title, dtype: int64

In [12]:
# for an array
titles_df.title.unique()

array(['Senior Engineer', 'Staff', 'Engineer', 'Senior Staff',
       'Assistant Engineer', 'Technique Leader', 'Manager'], dtype=object)

8. What is the oldest date in the to_date column?

In [13]:
titles_df.to_date.sort_values().head(1)

16064    1985-03-01
Name: to_date, dtype: object

9. What is the most recent date in the to_date column?

In [14]:
titles_df.to_date.sort_values(ascending=False).head()

443307    9999-01-01
191723    9999-01-01
191707    9999-01-01
191712    9999-01-01
191714    9999-01-01
Name: to_date, dtype: object

#### Exercise II

1. Copy the users and roles DataFrames from the examples above.

In [15]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [16]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


2. What is the result of using a right join on the DataFrames?

In [17]:
# right join
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='right')
            )

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


In [18]:
# left join
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='left')
            )

,id_x,name_x,role_id,id_y,name_y
0,1,bob,1.0,1.0,admin
1,2,joe,2.0,2.0,author
2,3,sally,3.0,3.0,reviewer
3,4,adam,3.0,3.0,reviewer
4,5,jane,NaN,NaN,NaN
5,6,mike,NaN,NaN,NaN


3. What is the result of using an outer join on the DataFrames?

In [19]:
# outer join
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer')
            )

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer')
    .drop(columns='role_id')
    .drop(columns='id')
            )
W get a KeyError: "['id'] not found in axis"

5. Load the mpg dataset from PyDataset.

In [21]:
mpg_df = data('mpg')
mpg_df.sample(5)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
83,ford,explorer 4wd,5.0,1999,8,auto(l4),4,13,17,r,suv
161,subaru,forester awd,2.5,1999,4,auto(l4),4,18,24,r,suv
136,lincoln,navigator 2wd,5.4,1999,8,auto(l4),r,11,16,p,suv
8,audi,a4 quattro,1.8,1999,4,manual(m5),4,18,26,p,compact
44,dodge,caravan 2wd,3.3,2008,6,auto(l4),f,11,17,e,minivan


6. Output and read the documentation for the mpg dataset.

In [22]:
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




7. How many rows and columns are in the dataset?

In [23]:
mpg_df.shape

(234, 11)

8. Check out your column names and perform any cleanup you may want on them.

In [24]:
mpg_df.rename(columns={'displ': 'display', 
                     'cyl': 'cylinder',
                     'trans': 'transmission',
                     'drv': 'drive',
                     'cty': 'city',
                     'hwy': 'highway',
                     'fl': 'fuel'}, inplace=True
            )

In [25]:
mpg_df.sample(5)

,manufacturer,model,display,year,cylinder,transmission,drive,city,highway,fuel,class
131,land rover,range rover,4.0,1999,8,auto(l4),4,11,15,p,suv
35,chevrolet,malibu,3.1,1999,6,auto(l4),f,18,26,r,midsize
32,chevrolet,k1500 tahoe 4wd,6.5,1999,8,auto(l4),4,14,17,d,suv
135,lincoln,navigator 2wd,5.4,1999,8,auto(l4),r,11,17,r,suv
149,nissan,maxima,3.0,1999,6,manual(m5),f,19,25,r,midsize


9. Display the summary statistics for the dataset.

In [26]:
mpg_df.describe()

,display,year,cylinder,city,highway
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


10. How many different manufacturers are there?

In [27]:
len(mpg_df.manufacturer.unique())

15

12. Create a column named mileage_difference like you did in the DataFrames exercises;
    this column should contain the difference between highway and city mileage for each car.


In [35]:
mpg_df['mileage_difference'] = mpg_df.highway - mpg_df.city
mpg_df.mileage_difference.sample(5)

163    6
220    7
101    8
164    6
83     4
Name: mileage_difference, dtype: int64

13. Create a column named average_mileage like you did in the DataFrames exercises; 
this is the mean of the city and highway mileage.

In [36]:
mpg_df['average_mileage'] = (mpg_df.highway + mpg_df.highway) / 2
mpg_df.average_mileage.sample(5)

21    20.0
53    19.0
27    25.0
45    22.0
18    23.0
Name: average_mileage, dtype: float64

14. Create a new column on the mpg dataset named is_automatic that holds boolean  values denoting whether the car has an automatic transmission.

In [41]:
mpg_df['is_automatic'] = mpg_df.transmission.str.contains('auto')
mpg_df.is_automatic.sample(5)

214    False
24     False
106     True
225     True
27      True
Name: is_automatic, dtype: bool

In [ ]:
# doing all three in one

# mpg = mpg.assign(mileage_difference = mpg.highway - mpg.city,
#                   average_mileage = (mpg.highway + mpg.city) / 2,
#                   is_automatic = mpg.transmission.str.startswith('a'))

15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

In [43]:
mpg_df

,manufacturer,model,display,year,cylinder,transmission,drive,city,highway,fuel,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,29.0,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,29.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,31.0,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,30.0,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,26.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,28.0,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,29.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,26.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,26.0,False


In [55]:
mpg_df.groupby('manufacturer').average_mileage.agg('mean').sort_values(ascending=False).head(1)

manufacturer
honda    32.555556
Name: average_mileage, dtype: float64

16. Do automatic or manual cars have better miles per gallon?

In [64]:
mpg_df.groupby(np.where(mpg_df.transmission.str.contains('manual'), 'manual', 'auto')).average_mileage.agg('mean')

auto      22.292994
manual    25.779221
Name: average_mileage, dtype: float64

In [75]:
# mpg_df.groupby('manufacturer')
# sns.relplot(x='transmission', y='is_automatic', hue = 'manufacturer', data=mpg_df)